# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Pre-Processing

#### Importing Libraries and creating a DataFrame from the Wiki Data

In [1]:
import pandas as pd
import numpy as np
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list=pd.read_html(url)
df=pd.DataFrame(df_list[0])

#### Dropping the rows which are Not assigned in Brorough

In [4]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Not assigned Neighbourhood are replaced with the vlaue in Borough

In [6]:
for index, row in df.iterrows():
    if df.loc[index,'Neighbourhood'] == 'Not assigned':
        df.loc[index,'Neighbourhood']=df.loc[index,'Borough']




#### Aggregating by unique Postcode and concatenating related Neighbourhood into string by comma seperated 

In [7]:
df=df.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

In [8]:
df.shape

(103, 3)

#### Reading Coordinates csv

In [9]:
Coordinates=pd.read_csv('Geospatial_Coordinates.csv')

#### Joining the Coordinates with our data set

In [10]:
df = pd.merge(df[['Postcode', 'Borough', 'Neighbourhood']],
                 Coordinates,
                 on=Coordinates['Postal Code'], 
                 how='inner')

#### How many boroguhs and neighbourhoods

In [19]:
#df=df.drop(['Postal Code','key_0'], axis=1)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)


The dataframe has 4 boroughs and 38 neighborhoods.


In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [26]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'WLIUWGSEAK1C0RJODW42EGRJUTYXK45RBOGNRXNN3B4IVMCK' # your Foursquare ID
CLIENT_SECRET = 'KIE2NWPDYGTWEFL3CJCYLHABMPNZCGNLFK3B4XCTGOBXMV0X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
# 		Client ID WLIUWGSEAK1C0RJODW42EGRJUTYXK45RBOGNRXNN3B4IVMCK
# 		Client Secret KIE2NWPDYGTWEFL3CJCYLHABMPNZCGNLFK3B4XCTGOBXMV0X

Your credentails:
CLIENT_ID: WLIUWGSEAK1C0RJODW42EGRJUTYXK45RBOGNRXNN3B4IVMCK
CLIENT_SECRET:KIE2NWPDYGTWEFL3CJCYLHABMPNZCGNLFK3B4XCTGOBXMV0X


## 1  Let's explore the first neighborhood in our dataframe.

In [29]:
toronto_data = df[df['Borough'] == 'East Toronto'].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [31]:
toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [32]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [33]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WLIUWGSEAK1C0RJODW42EGRJUTYXK45RBOGNRXNN3B4IVMCK&client_secret=KIE2NWPDYGTWEFL3CJCYLHABMPNZCGNLFK3B4XCTGOBXMV0X&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [35]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7acd5c351e3d13a80006c5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [40]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Glen Stewart Park,Park,43.675278,-79.294647
4,Upper Beaches,Neighborhood,43.680563,-79.292869
5,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


How many vensues returned by foursquare

In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [45]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


#### size of toronto_venies

In [47]:
toronto_venues.shape

(125, 7)

#### Let's check how many venues were returned for each neighborhood

In [48]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"Riverdale, The Danforth West",42,42,42,42,42,42
Studio District,40,40,40,40,40,40
The Beaches,6,6,6,6,6,6
"The Beaches West, India Bazaar",19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues

In [49]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 70 uniques categories.


## 3. Analyze Each Neighborhood

In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Arts & Crafts Store,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,...,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
toronto_onehot.shape

(125, 70)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Arts & Crafts Store,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,...,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.00000,0.000,0.055556,0.00000,0.000,0.000,0.000000,0.000000,...,0.055556,0.000000,0.000,0.055556,0.055556,0.00000,0.000,0.000000,0.000000,0.00000
1,"Riverdale, The Danforth West",0.023810,0.02381,0.000,0.000000,0.02381,0.000,0.000,0.000000,0.047619,...,0.023810,0.000000,0.000,0.000000,0.000000,0.02381,0.000,0.000000,0.000000,0.02381
2,Studio District,0.025000,0.05000,0.025,0.000000,0.05000,0.025,0.025,0.000000,0.025000,...,0.000000,0.025000,0.025,0.000000,0.000000,0.00000,0.025,0.000000,0.000000,0.00000
3,The Beaches,0.000000,0.00000,0.000,0.000000,0.00000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000,0.000000,0.000000,0.00000
4,"The Beaches West, India Bazaar",0.000000,0.00000,0.000,0.000000,0.00000,0.000,0.000,0.052632,0.000000,...,0.000000,0.052632,0.000,0.000000,0.000000,0.00000,0.000,0.052632,0.052632,0.00000


lets confirm new size

In [58]:
toronto_grouped.shape

(5, 70)

#### Let's print each neighborhood along with the top 5 most common venues

In [60]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2             Butcher  0.06
3          Smoke Shop  0.06
4          Skate Park  0.06


----Riverdale, The Danforth West----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----The Beaches----
               venue  freq
0        Coffee Shop  0.33
1               Park  0.17
2  Health Food Store  0.17
3                Pub  0.17
4        Yoga Studio  0.00


----The Beaches West, India Bazaar----
                  venue  freq
0             Pet Store  0.05
1          Burger Joint  0.05
2         Movie Theater  0.05
3  Fast Food Restauran

#### Let's put that into a *pandas* dataframe

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the  5 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant
1,"Riverdale, The Danforth West",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Trail,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store
2,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Gym / Fitness Center,Cheese Shop
3,The Beaches,Coffee Shop,Park,Health Food Store,Pub,Trail,Cosmetics Shop,Clothing Store,Comfort Food Restaurant,Comic Shop,Convenience Store
4,"The Beaches West, India Bazaar",Burrito Place,Pizza Place,Intersection,Italian Restaurant,Gym,Liquor Store,Burger Joint,Movie Theater,Brewery,Sushi Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [70]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 1, 0], dtype=int32)

##### Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [81]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Clust Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Clust Labels,Cluste Labels,Clusters2 Labels,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,1,1,1,1,Coffee Shop,Park,Health Food Store,Pub,Trail,Cosmetics Shop,Clothing Store,Comfort Food Restaurant,Comic Shop,Convenience Store
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,2,2,2,2,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Trail,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,0,0,0,0,Burrito Place,Pizza Place,Intersection,Italian Restaurant,Gym,Liquor Store,Burger Joint,Movie Theater,Brewery,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,0,0,0,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Gym / Fitness Center,Cheese Shop
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,0,0,0,0,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant


##### Visulaize the results

In [87]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Clust Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [ ]:
Cluster 1

In [89]:
toronto_merged.loc[toronto_merged['Clust Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Clust Labels,Cluste Labels,Clusters2 Labels,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,0,0,0,0,Burrito Place,Pizza Place,Intersection,Italian Restaurant,Gym,Liquor Store,Burger Joint,Movie Theater,Brewery,Sushi Restaurant
3,East Toronto,0,0,0,0,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Gym / Fitness Center,Cheese Shop
4,East Toronto,0,0,0,0,0,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant


Cluster 2

In [90]:
toronto_merged.loc[toronto_merged['Clust Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Clust Labels,Cluste Labels,Clusters2 Labels,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,1,1,1,1,Coffee Shop,Park,Health Food Store,Pub,Trail,Cosmetics Shop,Clothing Store,Comfort Food Restaurant,Comic Shop,Convenience Store


cluster 3

In [92]:
toronto_merged.loc[toronto_merged['Clust Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Clust Labels,Cluste Labels,Clusters2 Labels,Clusters Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,2,2,2,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Trail,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store
